In [1]:


import os
import csv
from datetime import datetime
import time
import re

In [ ]:
from lxml import html
from time import sleep
import requests

baseurl = 'https://www.metacritic.com'
starturl = '/browse/albums/release-date/new-releases/date'
url = baseurl+starturl

user_agent = 'Mozilla/5.0 (X11; Linux x86_64; rv:68.0) Gecko/20100101 Firefox/68.0'
headers = {'User-Agent': user_agent}
response = requests.get(url, headers=headers)

page = html.fromstring(response.content)

# find this: <td class="clamp-summary-wrap">

artists    = [artist.strip()[3:] for artist in page.xpath('//td[@class="clamp-summary-wrap"]/div/div[@class="artist"]/text()')]
albumname  = page.xpath('//td[@class="clamp-summary-wrap"]/a/h3/text()')
links      = page.xpath('//td[@class="clamp-summary-wrap"]/a/@href')
dates      = page.xpath('//td[@class="clamp-summary-wrap"]/div/span/text()')
metascores = page.xpath('//td[@class="clamp-summary-wrap"]/div/div[@class="clamp-metascore"]/a/div/text()')
userscores = page.xpath('//td[@class="clamp-summary-wrap"]/div/div[@class="clamp-userscore"]/a/div/text()')

recordlabels = []
count = 0
for link in links:
    response = requests.get(baseurl+link, headers=headers)
    page = html.fromstring(response.content)

    recordlabel = page.xpath('//span[contains(@class,"label") and contains(text(),"Record Label")]/following-sibling::span/text()')

    sleep(1)

    count += 1
    time.sleep(3)
    print(count)


maybe = zip(artists, albumname, recordlabels, dates, metascores, userscores)

print(list(maybe))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89


In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:

url_for_scrape = 'https://www.metacritic.com/browse/albums/release-date/new-releases/date'
user_agent = {'User-agent': 'Mozilla/5.0'}
# send response
response_score = requests.get(url_for_scrape, headers = user_agent)
# scrape website into variable to parse
soup_score = BeautifulSoup(response_score.text, 'html.parser')
# create list for dictionarys 
album_dicts = []
# create soup 
print(soup_score.find_all('td', class_='clamp-summary-wrap'))

In [ ]:
url_for_scrape = 'https://www.metacritic.com/browse/albums/release-date/new-releases/date'
user_agent = {'User-agent': 'Mozilla/5.0'}
# send response
response_score = requests.get(url_for_scrape, headers = user_agent)
# scrape website into variable to parse
soup_score = BeautifulSoup(response_score.text, 'html.parser')
# create list for dictionarys 
album_dicts = []
for item in soup_score.find_all('td', class_='clamp-summary-wrap'):
    
    album_dict = {}
    ###
    # scrape album name
    # regular expressions to prepare album and artist names for 
    # search in spotify DB
    ####
    album_element = item.find('a', class_= 'title')
    print(album_element)
    print(album_element.text)
    print(album_element['href'])
    browser.find_by_css('a'.click())
#     browser.click_link_by_text(album_element['href'])
    break
    al = re.sub(r'[^-A-Za-z0-9!áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]+', '', album_raw)
    album_clean = re.sub(' +', ' ', al)
    album_dict['album']= album_clean
    ###
    # scrape artist name and strip white space and extra characters
    ###
    artist_raw = item.find('div', class_='artist').text.strip().lstrip('by ')
    ar = re.sub(r'[^-A-Za-z0-9!áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]+', '', artist_raw)
    artist_clean= re.sub(' +', ' ', ar)
    album_dict['artist']=artist_clean

    ###        
    # scrape date
    #sqlite doesn't support datetime objects date column will be string
    #make an objec of date to extract week num and year
    ###
    date_string = (item.find('div', class_='clamp-details').find('span').text)
    date_obj = datetime.strptime(date_string, '%B %d, %Y')
    year, album_week_num, day_of_week = date_obj.isocalendar()
    album_dict['date']=date_string
    album_dict['year'] = year
    album_dict['week_num']=album_week_num
    ###
    # Handle for varaitions in classes for critic name by pattern matching with regular expression.
    # then scrape critic and user scores
    ###
    meta_critic_pattern = re.compile('^metascore_w large')
    meta_user_pattern = re.compile('^metascore_w user')
    album_dict['meta_score']=int(item.find('div', class_= meta_critic_pattern).text)
    user_string = (item.find('div', class_= meta_user_pattern).text)
    ###
    # Handle for variations in classes for user by filtering out strings from scores to and casting to ints
    ###
    if user_string == 'tbd':
        album_dict['user_score']=user_string
    else:
        user_score = int(float(user_string)*10)
        album_dict['user_score']=user_score
    album_dicts.append(album_dict)
